In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
green_tripdata = pd.read_csv("green_tripdata_2019-09.csv")
taxi_zone = pd.read_csv("taxi+_zone_lookup.csv")

/var/folders/r_/c2lk8b9s5qb2tpb3ql4jzh100000gn/T/ipykernel_56269/2991038093.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  green_tripdata = pd.read_csv("green_tripdata_2019-09.csv")


In [118]:
green_tripdata.shape, taxi_zone.shape

((449063, 20), (265, 4))

In [3]:
# time conversion

date_columns = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']

green_tripdata[date_columns] = green_tripdata[date_columns].apply(pd.to_datetime)

In [2]:
# db connection

USERNAME = 'keniwo'
PW = '2000'
HOST = 'localhost'
PORT = '5432'
DBNAME = ''

engine = create_engine(f'postgresql://{USERNAME}:{PW}@{HOST}:{PORT}/{DBNAME}')
connection = engine.connect()

In [68]:
slice = green_tripdata[:100] 

# let's learn con, name if_exists -- these 3 params first)
slice.to_sql(name='green_tripdata', con=engine, if_exists='replace')

100

In [111]:
slice2 = taxi_zone[:100]
slice2.to_sql(name='taxi_zone', con=engine, if_exists='replace')

100

In [73]:
slice.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1.0,65,189,5.0,2.00,10.5,0.5,0.5,2.36,0.0,NaN,0.3,14.16,1.0,1.0,0.0
1,2.0,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1.0,97,225,5.0,3.20,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2.0,1.0,0.0
2,2.0,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1.0,37,61,5.0,2.99,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2.0,1.0,0.0
3,2.0,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1.0,145,112,1.0,1.73,7.5,0.5,0.5,1.50,0.0,NaN,0.3,10.30,1.0,1.0,0.0
4,2.0,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1.0,112,198,1.0,3.42,14.0,0.5,0.5,3.06,0.0,NaN,0.3,18.36,1.0,1.0,0.0


In [87]:
from sqlalchemy import MetaData, select

metadata = MetaData()
metadata.reflect(bind=engine)

# create MetaData() and fetch Table object
green_tripdata_df = metadata.tables['green_tripdata']
green_tripdata_df.c.keys()[:4]

['index', 'VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime']

In [117]:
#query example

query = select(green_tripdata_df).where(green_tripdata_df.c.lpep_pickup_datetime > '2019-09-01 00:20:00')

result = connection.execute(query)
pd.DataFrame(result, columns=result.keys()) \
    .drop(columns=['index']).head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1.0,97,225,5.0,3.20,12.0,0.5,0.5,0.00,0.00,None,0.3,13.30,2.0,1.0,0.0
1,2.0,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1.0,37,61,5.0,2.99,12.0,0.5,0.5,0.00,0.00,None,0.3,13.30,2.0,1.0,0.0
2,2.0,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1.0,145,112,1.0,1.73,7.5,0.5,0.5,1.50,0.00,None,0.3,10.30,1.0,1.0,0.0
3,2.0,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1.0,112,198,1.0,3.42,14.0,0.5,0.5,3.06,0.00,None,0.3,18.36,1.0,1.0,0.0
4,2.0,2019-09-01 00:55:37,2019-09-01 01:17:02,N,1.0,7,41,1.0,6.26,21.0,0.5,0.5,0.00,6.12,None,0.3,28.42,2.0,1.0,0.0


Gonna import the whole dataset into the db by iteration

!!! Notice how to create a iter df by assigning `iterator` and `chunksize` params.

In [3]:
# create iter df
green_tripdata_iter = pd.read_csv("green_tripdata_2019-09.csv", iterator=True, chunksize=10000)
taxi_zone = pd.read_csv("taxi+_zone_lookup.csv")

from time import time
import logging

logging.basicConfig(
    filename='./data/data-ingestion.log', 
    filemode='w',  # Set filemode to 'w' for write mode (overwrite)
    level=logging.INFO, 
    format='%(asctime)s - %(message)s', # timestamp of the log message + message body
    datefmt='%H:%M:%S',
)

logging.info('Start logging...')

for i, iter in enumerate(green_tripdata_iter):
    action = 'append' if i else 'replace'

    date_columns = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
    iter[date_columns] = iter[date_columns].apply(pd.to_datetime)
    
    start_time = time()
    iter.to_sql(name='green_tripdata', con=engine, if_exists=action)
    end_time = time()
    time_span = end_time - start_time
    
    logging.info(f'Done with the chunk {i}. Took {time_span} seconds.')
    
taxi_zone.to_sql(name='taxi_zone', con=engine, if_exists='replace')
logging.info(f'Done with the taxi_zone data.')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/keniwo/ws/de-notes/hw1/data/data-ingestion.log'

In [12]:
!jupyter nbconvert --to script data-ingesting.ipynb

[NbConvertApp] Converting notebook data-ingestion.ipynb to script
[NbConvertApp] Writing 2934 bytes to data-ingestion.py
